# Capstone Project: The Battle of Neighborhood
###  Recommending the Most Valuable Venue for Homebuyers in London

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction <a name="introduction"></a>

### Background
According Working Paper 72 published by London Government, London’s house prices are relatively high comparing to other area in the UK, and have been rising at a fast rate recent years. Meanwhile, housing markets in London have witnessed a number of ups and downs, with volatile house prices in London tending to amplify changes in national house prices. London has experienced several episodes of real house price deflation. From the patterns of previous cycles, it is hard to simply tell whether house prices are approaching a new peak or a downward adjustment.

Recently as a result of the pandemic, increasing unemployment rate, wage cuts, business failures, and job uncertainty also brought huge impact to the house market in London. According to the Nationwide, house prices fell 1.7% in May from the previous month, the largest monthly fall for 11 years, due to the headwinds of Covid-19.

### Business Problem
As a matter of fact, it is of great necessity to integrate data science and machine learning technics with the study of London house market in order to assist clients making sensible and efficient choices.
How could we provide suggestions to our clients based on the study of neighborhoods for either settling or investing is our main business problem. We will recommend profitable venues according to amenities and infrastructures surrounded.

## Data <a name="data"></a>

London properties monthly updated price paid data is provided from the HM Land Registry.
Data link: http://landregistry.data.gov.uk/

After finding the list of neighborhoods, we then connect to the Foursquare API to gather information about venues inside each and every neighborhood. 
FourSquare API interface is a location data provider with information about venues. Data retrieved includes venue name, latitude, longitude, category, neighborhood and neighborhood location.  

### Properties Data

Let's first download the properties' data from the HM Land Registry website:
#### Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/nbuser/anaconda3_501

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    conda-package-handling-1.6.0|   py36h8c4c3a4_2         947 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    conda-4.8.2                |           py36_0         3.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following

liblief-0.10.1       | 2.0 MB    | ##################################### | 100% 
folium-0.5.0         | 45 KB     | ##################################### | 100% 
scikit-image-0.14.2  | 24.0 MB   | ##################################### | 100% 
scikit-learn-0.20.1  | 5.7 MB    | ##################################### | 100% 
soupsieve-2.0.1      | 56 KB     | ##################################### | 100% 
ripgrep-12.1.1       | 1.8 MB    | ##################################### | 100% 
sphinxcontrib-htmlhe | 27 KB     | ##################################### | 100% 
ca-certificates-2020 | 145 KB    | ##################################### | 100% 
sphinxcontrib-devhel | 22 KB     | ##################################### | 100% 
conda-4.8.4          | 3.1 MB    | ##################################### | 100% 
branca-0.4.1         | 26 KB     | ##################################### | 100% 
sphinxcontrib-appleh | 28 KB     | ##################################### | 100% 
sphinxcontrib-jsmath | 7 KB 

In [2]:
import requests
import xml

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

#### Retrieve Data from source
Here we use the data in csv file which containing transfer information in 2019.

In [3]:
df = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv")

df.shape

(972285, 16)

In [4]:
df.head(5)

,{8CAC1318-AC2F-0253-E053-6B04A8C08E51},165950,2019-05-28 00:00,BS22 7FP,T,N,F,32,Unnamed: 8,KELSTON GARDENS,Unnamed: 10,WESTON-SUPER-MARE,NORTH SOMERSET,NORTH SOMERSET.1,A,A.1
0,{8CAC1318-AC31-0253-E053-6B04A8C08E51},119500,2019-05-22 00:00,TA8 2EY,F,N,L,"GROVE HOUSE, 58",FLAT 1,BERROW ROAD,NaN,BURNHAM-ON-SEA,SEDGEMOOR,SOMERSET,A,A
1,{8CAC1318-AC33-0253-E053-6B04A8C08E51},215000,2019-06-21 00:00,TA22 9DH,T,N,F,5,NaN,WEIR HEAD,NaN,DULVERTON,SOMERSET WEST AND TAUNTON,SOMERSET,A,A
2,{8CAC1318-AC35-0253-E053-6B04A8C08E51},242500,2019-05-01 00:00,BS20 7BP,F,N,L,50,NaN,LOWER BURLINGTON ROAD,PORTISHEAD,BRISTOL,NORTH SOMERSET,NORTH SOMERSET,A,A
3,{8CAC1318-AC36-0253-E053-6B04A8C08E51},318000,2019-05-09 00:00,BA3 2RW,T,N,F,4,NaN,BUSHY COMBE,MIDSOMER NORTON,RADSTOCK,BATH AND NORTH EAST SOMERSET,BATH AND NORTH EAST SOMERSET,A,A
4,{8CAC1318-AC37-0253-E053-6B04A8C08E51},215000,2019-05-17 00:00,BS24 7HZ,S,N,F,9,NaN,OSMOND ROAD,NaN,WESTON-SUPER-MARE,NORTH SOMERSET,NORTH SOMERSET,A,A


#### Data preprocessing
We need to process the dataframe for further usage. Here we aim to group the venues by Street assigned in the dataframe and look into the average price of each street. 

Targetting housebuyers' price expectation, upper limit and lower limit of price can be set to narrow down the venues we are looking at.

Preprocessing the data to finally obtain a dataframe containing the following information:
* Street Name
* Average Price of Venues
* Location Information

**1. Rename the column names**

In [5]:
# Assign meaningful column names
df.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']
df.head()

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,City,District,County,PPD_Cat_Type,Record_Status
0,{8CAC1318-AC31-0253-E053-6B04A8C08E51},119500,2019-05-22 00:00,TA8 2EY,F,N,L,"GROVE HOUSE, 58",FLAT 1,BERROW ROAD,NaN,BURNHAM-ON-SEA,SEDGEMOOR,SOMERSET,A,A
1,{8CAC1318-AC33-0253-E053-6B04A8C08E51},215000,2019-06-21 00:00,TA22 9DH,T,N,F,5,NaN,WEIR HEAD,NaN,DULVERTON,SOMERSET WEST AND TAUNTON,SOMERSET,A,A
2,{8CAC1318-AC35-0253-E053-6B04A8C08E51},242500,2019-05-01 00:00,BS20 7BP,F,N,L,50,NaN,LOWER BURLINGTON ROAD,PORTISHEAD,BRISTOL,NORTH SOMERSET,NORTH SOMERSET,A,A
3,{8CAC1318-AC36-0253-E053-6B04A8C08E51},318000,2019-05-09 00:00,BA3 2RW,T,N,F,4,NaN,BUSHY COMBE,MIDSOMER NORTON,RADSTOCK,BATH AND NORTH EAST SOMERSET,BATH AND NORTH EAST SOMERSET,A,A
4,{8CAC1318-AC37-0253-E053-6B04A8C08E51},215000,2019-05-17 00:00,BS24 7HZ,S,N,F,9,NaN,OSMOND ROAD,NaN,WESTON-SUPER-MARE,NORTH SOMERSET,NORTH SOMERSET,A,A


**2. Select column City which is only London**

In [6]:
df_ld = df.query("City == 'LONDON'")
df_ld.head()

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,City,District,County,PPD_Cat_Type,Record_Status
86,{8CAC1318-AFF3-0253-E053-6B04A8C08E51},1100000,2019-06-07 00:00,SW19 8BN,T,N,F,30,NaN,ASHEN GROVE,NaN,LONDON,MERTON,GREATER LONDON,A,A
88,{8CAC1318-AFF5-0253-E053-6B04A8C08E51},905000,2019-06-20 00:00,SW19 8BH,T,N,F,35,NaN,DURNSFORD AVENUE,NaN,LONDON,MERTON,GREATER LONDON,A,A
95,{8CAC1318-AFFC-0253-E053-6B04A8C08E51},1440000,2019-06-07 00:00,SW13 8DL,T,N,F,29,NaN,MERTHYR TERRACE,NaN,LONDON,RICHMOND UPON THAMES,GREATER LONDON,A,A
167,{8CAC1318-B38E-0253-E053-6B04A8C08E51},402500,2019-06-14 00:00,SW16 4PB,S,N,F,9,NaN,POLLARDS WOOD ROAD,NaN,LONDON,CROYDON,GREATER LONDON,A,A
292,{8F1B26BD-4595-53DB-E053-6C04A8C03649},340000,2019-07-15 00:00,N1 9BT,F,N,L,75,FLAT 4,CALEDONIAN ROAD,NaN,LONDON,ISLINGTON,GREATER LONDON,A,A


**3. Format transfer date**

In [7]:
df_ld['Date_Transfer'] = df_ld['Date_Transfer'].apply(pd.to_datetime)

# Sort by Date
df_ld.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)
df_ld.head()

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,City,District,County,PPD_Cat_Type,Record_Status
730629,{A2479555-A33A-74C7-E053-6B04A8C0887D},670000,2019-12-31,SW18 1UT,F,Y,L,8,APARTMENT 1,OSIERS ROAD,NaN,LONDON,WANDSWORTH,GREATER LONDON,A,A
730613,{A2479555-A326-74C7-E053-6B04A8C0887D},600000,2019-12-31,SW18 1UX,F,Y,L,12,APARTMENT 46,OSIERS ROAD,NaN,LONDON,WANDSWORTH,GREATER LONDON,A,A
730633,{A2479555-A347-74C7-E053-6B04A8C0887D},520000,2019-12-31,SW18 1UX,F,Y,L,12,APARTMENT 7,OSIERS ROAD,NaN,LONDON,WANDSWORTH,GREATER LONDON,A,A
730636,{A2479555-A34B-74C7-E053-6B04A8C0887D},600000,2019-12-31,SW18 1UT,F,Y,L,8,APARTMENT 12,OSIERS ROAD,NaN,LONDON,WANDSWORTH,GREATER LONDON,A,A
819258,{9FF0D96A-38F5-11ED-E053-6C04A8C06383},600000,2019-12-31,SW18 1UX,F,Y,L,"FLAT 45, 12",NaN,OSIERS ROAD,NaN,LONDON,WANDSWORTH,GREATER LONDON,A,A


**4. Group by Street and calculate street average price**

In [8]:
streets = df_ld['Street'].unique().tolist()
df_grp_price = df_ld.groupby(['Street'])['Price'].mean().reset_index()

df_grp_price.columns = ['Street', 'Avg_Price']

Depend on the Client's Budget to set upper limit and lower limit that fits the inquiry

In [9]:
df_price_limit = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

df_price_limit.head()

,Street,Avg_Price
171,ALBERT EMBANKMENT,2.283829e+06
186,ALBION SQUARE,2.292500e+06
258,ALLEYN PARK,2.283095e+06
455,ARGYLE STREET,2.300000e+06
499,ARTESIAN ROAD,2.462500e+06


**5. Use geolocator to obtain the latitude and longitutde for each street **

In [10]:
for index, item in df_price_limit.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 171
item: Street       ALBERT EMBANKMENT
Avg_Price          2.28383e+06
Name: 171, dtype: object
item.Street only: ALBERT EMBANKMENT
index: 186
item: Street       ALBION SQUARE
Avg_Price       2.2925e+06
Name: 186, dtype: object
item.Street only: ALBION SQUARE
index: 258
item: Street       ALLEYN PARK
Avg_Price     2.2831e+06
Name: 258, dtype: object
item.Street only: ALLEYN PARK
index: 455
item: Street       ARGYLE STREET
Avg_Price          2.3e+06
Name: 455, dtype: object
item.Street only: ARGYLE STREET
index: 499
item: Street       ARTESIAN ROAD
Avg_Price       2.4625e+06
Name: 499, dtype: object
item.Street only: ARTESIAN ROAD
index: 536
item: Street       ASHCHURCH GROVE
Avg_Price          2.425e+06
Name: 536, dtype: object
item.Street only: ASHCHURCH GROVE
index: 680
item: Street       AYNHOE ROAD
Avg_Price      2.475e+06
Name: 680, dtype: object
item.Street only: AYNHOE ROAD
index: 989
item: Street       BEDALE STREET
Avg_Price       2.3446e+06
Name: 989, dtype: object
it

index: 9047
item: Street       NORRICE LEA
Avg_Price      2.355e+06
Name: 9047, dtype: object
item.Street only: NORRICE LEA
index: 9152
item: Street       OAK HILL PARK MEWS
Avg_Price               2.4e+06
Name: 9152, dtype: object
item.Street only: OAK HILL PARK MEWS
index: 9252
item: Street       OLD KENT ROAD
Avg_Price      2.24288e+06
Name: 9252, dtype: object
item.Street only: OLD KENT ROAD
index: 9301
item: Street       ONSLOW CRESCENT
Avg_Price            2.4e+06
Name: 9301, dtype: object
item.Street only: ONSLOW CRESCENT
index: 9337
item: Street       ORMONDE GATE
Avg_Price        2.35e+06
Name: 9337, dtype: object
item.Street only: ORMONDE GATE
index: 9418
item: Street       PADDINGTON STREET
Avg_Price          2.39321e+06
Name: 9418, dtype: object
item.Street only: PADDINGTON STREET
index: 9433
item: Street       PALACE COURT
Avg_Price     2.27367e+06
Name: 9433, dtype: object
item.Street only: PALACE COURT
index: 9512
item: Street       PARKE ROAD
Avg_Price      2.25e+06
Nam

In [11]:
geolocator = Nominatim(user_agent="http")

In [13]:
# add in latitude and longitude column
df_price_limit['Latitude'] = df_price_limit['Street'].apply(geolocator.geocode).apply(lambda x: x.latitude)
df_price_limit['Longitude'] = df_price_limit['Street'].apply(geolocator.geocode).apply(lambda x: x.longitude)

print(df_price_limit.shape)
df_p = df_price_limit
df_p.head()

(144, 4)


,Street,Avg_Price,Latitude,Longitude
171,ALBERT EMBANKMENT,2.283829e+06,51.493213,-0.121361
186,ALBION SQUARE,2.292500e+06,-41.273758,173.289393
258,ALLEYN PARK,2.283095e+06,51.492742,-0.371967
455,ARGYLE STREET,2.300000e+06,22.319036,114.167841
499,ARTESIAN ROAD,2.462500e+06,41.725304,-88.205529


### Map of London

In [14]:
address = 'London,UK'

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The Geograpical Co-ordinate of Seattle,Washington are {}, {}.'.format(latitude, longitude))

The Geograpical Co-ordinate of Seattle,Washington are 51.5073219, -0.1276474.


In [15]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, street in zip(df_p['Latitude'], df_p['Longitude'], df_p['Street']):
    label = '{}'.format(street)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

## Methodology<a name="data"></a>

For the methodology sector, the main goal is to analysis the data and recommend the most valuable venues for the customers based on the surrounding infrastructure and amenities, which retrieved from the Foursquare API.

K-means clustering is used to analyze the venues, given the fact that it is efficient in terms of computational cost and easy to implement when studying larget data set.

### Foursquare Credentials 

In [16]:
CLIENT_ID = 'ZUG035NNNJK3W5UR2CECWBHTBDPYUZRIPRHBJQU31WEKVYLK' # my Foursquare ID
CLIENT_SECRET = 'G4IL2IN232SOLGAKEX2ZFRWWCRTFNH1E1TKRQ30GJ20YUNZS' # my Foursquare Secret
VERSION = '20200604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZUG035NNNJK3W5UR2CECWBHTBDPYUZRIPRHBJQU31WEKVYLK
CLIENT_SECRET: G4IL2IN232SOLGAKEX2ZFRWWCRTFNH1E1TKRQ30GJ20YUNZS


### Nearby Venues

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
# get the venues for each street in the dataframe
london_venues = getNearbyVenues(names=df_p['Street'],
                                   latitudes=df_p['Latitude'],
                                   longitudes=df_p['Longitude']
                                  )

ALBERT EMBANKMENT
ALBION SQUARE
ALLEYN PARK
ARGYLE STREET
ARTESIAN ROAD
ASHCHURCH GROVE
AYNHOE ROAD
BEDALE STREET
BEECHWOOD AVENUE
BETTERTON STREET
BETTRIDGE ROAD
BILLING ROAD
BLENHEIM CRESCENT
BOURDON STREET
BRADFIELD ROAD
BROMPTON PLACE
CAITHNESS ROAD
CAMBRIDGE TERRACE
CAMDEN SQUARE
CAMPDEN HILL ROAD
CANONBURY PARK SOUTH
CARLISLE PLACE
CARLTON GARDENS
CARLTON ROAD
CHALCOT SQUARE
CHELSEA EMBANKMENT
CHENISTON GARDENS
CHESHAM MEWS
CHEVENING ROAD
COLLINGHAM ROAD
COULTER ROAD
COURT LANE GARDENS
COURTHOPE ROAD
COURTNELL STREET
CRAWFORD MEWS
CREDITON HILL
DARTMOUTH PARK AVENUE
DE VERE GARDENS
DEEPDALE
DEER PARK ROAD
DEVEREUX LANE
DEVONSHIRE MEWS WEST
DOVER STREET
DOWNSIDE CRESCENT
DUDLEY ROAD
DUKES LANE
EATON TERRACE MEWS
ECCLESTON MEWS
ECCLESTON SQUARE
ELLERBY STREET
ELVASTON PLACE
ENNISMORE GARDENS MEWS
ESSEX STREET
EVERSLEY CRESCENT
EYNELLA ROAD
FENCHURCH STREET
FLORAL STREET
FOUNTAYNE ROAD
FOURNIER STREET
FRISTON STREET
GERTRUDE STREET
GLOUCESTER PLACE MEWS
GORDON COTTAGES
GORST ROAD
GR

In [19]:
print(london_venues.shape)
london_venues.head()

(3688, 7)


,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBERT EMBANKMENT,51.493213,-0.121361,"The Garden Cafe, Lambeth Palace",51.495210,-0.119962,Café
1,ALBERT EMBANKMENT,51.493213,-0.121361,Newport Street Gallery,51.493466,-0.117040,Art Gallery
2,ALBERT EMBANKMENT,51.493213,-0.121361,Plaza On The River Club And Residence,51.491529,-0.121371,Hotel
3,ALBERT EMBANKMENT,51.493213,-0.121361,Tamesis Dock,51.491928,-0.121934,Bar
4,ALBERT EMBANKMENT,51.493213,-0.121361,Park Plaza London Riverbank,51.491478,-0.122177,Hotel


In [23]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))
print(london_venues.shape)
london_venues.groupby('Street').count().head()

There are 318 uniques categories.
(3688, 7)


,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBERT EMBANKMENT,53,53,53,53,53,53
ALBION SQUARE,26,26,26,26,26,26
ALLEYN PARK,6,6,6,6,6,6
ARGYLE STREET,87,87,87,87,87,87
ARTESIAN ROAD,8,8,8,8,8,8


### One Hot Encoding of Features

In [24]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
london_onehot['Street'] = london_venues['Street'] 

# move street column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]
london_onehot.head()

,Street,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Terminal,American Restaurant,Antique Shop,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,ALBERT EMBANKMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBERT EMBANKMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBERT EMBANKMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBERT EMBANKMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBERT EMBANKMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
london_grouped = london_onehot.groupby('Street').mean().reset_index()
london_grouped.head()

,Street,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Terminal,American Restaurant,Antique Shop,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,ALBERT EMBANKMENT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ALBION SQUARE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ALLEYN PARK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ARGYLE STREET,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.011494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ARTESIAN ROAD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# top 5 venues valued most
num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBERT EMBANKMENT----
         venue  freq
0   Restaurant  0.13
1        Hotel  0.11
2         Café  0.11
3          Pub  0.08
4  Pizza Place  0.04


----ALBION SQUARE----
         venue  freq
0         Café  0.19
1          Pub  0.08
2          Bar  0.08
3  Coffee Shop  0.08
4   Restaurant  0.08


----ALLEYN PARK----
               venue  freq
0                Pub  0.50
1  Indian Restaurant  0.17
2             Lawyer  0.17
3         Playground  0.17
4                ATM  0.00


----ARGYLE STREET----
                 venue  freq
0       Cha Chaan Teng  0.07
1         Dessert Shop  0.06
2         Noodle House  0.06
3                Hotel  0.05
4  Sporting Goods Shop  0.05


----ARTESIAN ROAD----
              venue  freq
0     Jewelry Store  0.12
1  Sushi Restaurant  0.12
2       Auto Garage  0.12
3          Pharmacy  0.12
4    Pilates Studio  0.12


----ASHCHURCH GROVE----
                      venue  freq
0                       Pub  0.15
1             Grocery Store  0.15
2  Medit

                venue  freq
0                Café  0.08
1  Italian Restaurant  0.08
2               Hotel  0.07
3         Coffee Shop  0.05
4              Bakery  0.05


----ECCLESTON MEWS----
         venue  freq
0        Hotel  0.08
1         Café  0.08
2   Restaurant  0.08
3  Coffee Shop  0.05
4    Gastropub  0.05


----ECCLESTON SQUARE----
                venue  freq
0               Hotel  0.11
1                 Pub  0.07
2  Italian Restaurant  0.07
3              Bakery  0.05
4                Café  0.05


----ELLERBY STREET----
                venue  freq
0                Café  0.18
1  Athletics & Sports  0.09
2                Park  0.09
3        Tennis Court  0.05
4         Coffee Shop  0.05


----ELVASTON PLACE----
                      venue  freq
0                     Hotel  0.14
1            Science Museum  0.09
2                   Exhibit  0.08
3  Mediterranean Restaurant  0.06
4                      Café  0.05


----ENNISMORE GARDENS MEWS----
                venue  freq
0  

                           venue  freq
0                            Pub  0.67
1               Botanical Garden  0.33
2                            ATM  0.00
3           Other Great Outdoors  0.00
4  Paper / Office Supplies Store  0.00


----NIGHTINGALE SQUARE----
            venue  freq
0             Pub  0.36
1  Farmers Market  0.09
2      Steakhouse  0.09
3         Brewery  0.09
4     Supermarket  0.09


----NORRICE LEA----
                   venue  freq
0                   Park  0.50
1  Outdoors & Recreation  0.25
2    Arts & Crafts Store  0.25
3                    ATM  0.00
4  Performing Arts Venue  0.00


----OAK HILL PARK MEWS----
                venue  freq
0                 Pub  0.12
1                Café  0.10
2              Bakery  0.10
3  Italian Restaurant  0.08
4      Ice Cream Shop  0.05


----OLD KENT ROAD----
            venue  freq
0             Bar  0.12
1           Hotel  0.12
2      Public Art  0.12
3            Café  0.12
4  Breakfast Spot  0.12


----ONSLOW CRESCEN

                           venue  freq
0              Convenience Store   1.0
1                            ATM   0.0
2              Outdoor Sculpture   0.0
3                           Park   0.0
4  Paper / Office Supplies Store   0.0


----WEST HILL PARK----
                           venue  freq
0                            Pub  0.50
1             English Restaurant  0.25
2                    Pizza Place  0.25
3  Paper / Office Supplies Store  0.00
4                         Palace  0.00


----WEST SQUARE----
                venue  freq
0                 Pub  0.15
1         Coffee Shop  0.15
2    Stationery Store  0.08
3  Mexican Restaurant  0.08
4      Sandwich Place  0.08


----WEST TEMPLE SHEEN----
                       venue  freq
0                        Pub   0.4
1  Middle Eastern Restaurant   0.2
2               Tennis Court   0.2
3                       Park   0.2
4                        ATM   0.0


----WHITE HART LANE----
                 venue  freq
0                  Pub  

### Most Common Venues on the Street

In [31]:
# definde the function that returns the most common venue
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create dataframe
common_venues_sorted = pd.DataFrame(columns=columns)
common_venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    common_venues_sorted.iloc[ind, 1:] = most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [34]:
print(common_venues_sorted.shape)
common_venues_sorted.head()

(130, 11)


,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBERT EMBANKMENT,Restaurant,Café,Hotel,Pub,Park,Art Gallery,Plaza,Pizza Place,Coffee Shop,Breakfast Spot
1,ALBION SQUARE,Café,Pub,Bar,Indian Restaurant,Restaurant,Coffee Shop,Park,Seafood Restaurant,Fish & Chips Shop,New American Restaurant
2,ALLEYN PARK,Pub,Indian Restaurant,Playground,Lawyer,Farmers Market,English Restaurant,Event Service,Event Space,Exhibit,Fabric Shop
3,ARGYLE STREET,Cha Chaan Teng,Noodle House,Dessert Shop,Hotel,Sporting Goods Shop,Hong Kong Restaurant,Vegetarian / Vegan Restaurant,Cantonese Restaurant,Bakery,Toy / Game Store
4,ARTESIAN ROAD,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Jewelry Store,Auto Garage,Pizza Place,Pilates Studio,Pharmacy,Event Space,Exhibit


### K-Means Clustering
After inspecting the data we obtained, we then apply k-means clustering approach with 5 clusters to study the data.
#### 1. Create clustering dataframe

In [37]:
k = 5
london_grouped_clustering = df_p.drop('Street', 1)
kmeans = KMeans(n_clusters=k, random_state=0).fit(london_grouped_clustering)
kmeans.labels_

array([3, 3, 3, 3, 1, 4, 1, 2, 1, 4, 4, 4, 2, 0, 0, 4, 0, 4, 4, 3, 2, 3,
       1, 3, 2, 0, 3, 3, 1, 1, 4, 4, 0, 4, 1, 3, 4, 0, 1, 4, 3, 4, 1, 2,
       3, 2, 4, 0, 1, 4, 2, 0, 1, 0, 3, 1, 4, 1, 4, 4, 3, 1, 0, 3, 0, 4,
       0, 4, 3, 0, 0, 4, 4, 0, 4, 1, 1, 0, 0, 2, 3, 1, 0, 3, 2, 3, 3, 4,
       2, 0, 3, 3, 1, 0, 1, 2, 4, 0, 4, 2, 4, 3, 0, 2, 0, 3, 3, 3, 0, 2,
       4, 3, 0, 4, 2, 4, 3, 0, 4, 2, 4, 0, 1, 3, 0, 2, 2, 0, 0, 1, 0, 0,
       4, 0, 3, 4, 4, 4, 3, 1, 3, 0, 3, 4], dtype=int32)

In [44]:
# check if using the correct dataframe
london_grouped_clustering=df_p
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude,Cluster Labels
171,ALBERT EMBANKMENT,2.283829e+06,51.493213,-0.121361,3
186,ALBION SQUARE,2.292500e+06,-41.273758,173.289393,3
258,ALLEYN PARK,2.283095e+06,51.492742,-0.371967,3
455,ARGYLE STREET,2.300000e+06,22.319036,114.167841,3
499,ARTESIAN ROAD,2.462500e+06,41.725304,-88.205529,1


In [54]:
common_venues_sorted.head()

,Cluster Labels,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,ALBERT EMBANKMENT,Restaurant,Café,Hotel,Pub,Park,Art Gallery,Plaza,Pizza Place,Coffee Shop,Breakfast Spot
1,2,ALBION SQUARE,Café,Pub,Bar,Indian Restaurant,Restaurant,Coffee Shop,Park,Seafood Restaurant,Fish & Chips Shop,New American Restaurant
2,3,ALLEYN PARK,Pub,Indian Restaurant,Playground,Lawyer,Farmers Market,English Restaurant,Event Service,Event Space,Exhibit,Fabric Shop
3,2,ARGYLE STREET,Cha Chaan Teng,Noodle House,Dessert Shop,Hotel,Sporting Goods Shop,Hong Kong Restaurant,Vegetarian / Vegan Restaurant,Cantonese Restaurant,Bakery,Toy / Game Store
4,2,ARTESIAN ROAD,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Jewelry Store,Auto Garage,Pizza Place,Pilates Studio,Pharmacy,Event Space,Exhibit


In [55]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
common_venues_sorted = common_venues_sorted.drop(columns = 'Cluster Labels')

In [56]:
london_grouped_clustering = london_grouped_clustering.join(common_venues_sorted.set_index('Street'), on='Street')

In [57]:
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
171,ALBERT EMBANKMENT,2.283829e+06,51.493213,-0.121361,3,Restaurant,Café,Hotel,Pub,Park,Art Gallery,Plaza,Pizza Place,Coffee Shop,Breakfast Spot
186,ALBION SQUARE,2.292500e+06,-41.273758,173.289393,3,Café,Pub,Bar,Indian Restaurant,Restaurant,Coffee Shop,Park,Seafood Restaurant,Fish & Chips Shop,New American Restaurant
258,ALLEYN PARK,2.283095e+06,51.492742,-0.371967,3,Pub,Indian Restaurant,Playground,Lawyer,Farmers Market,English Restaurant,Event Service,Event Space,Exhibit,Fabric Shop
455,ARGYLE STREET,2.300000e+06,22.319036,114.167841,3,Cha Chaan Teng,Noodle House,Dessert Shop,Hotel,Sporting Goods Shop,Hong Kong Restaurant,Vegetarian / Vegan Restaurant,Cantonese Restaurant,Bakery,Toy / Game Store
499,ARTESIAN ROAD,2.462500e+06,41.725304,-88.205529,1,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Jewelry Store,Auto Garage,Pizza Place,Pilates Studio,Pharmacy,Event Space,Exhibit


#### 2. Create map of clustering

In [58]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], 
                                  london_grouped_clustering['Longitude'], 
                                  london_grouped_clustering['Street'], 
                                  london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Result

In [60]:
df1 = london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, 
                              london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]]
df2 = london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, 
                                    london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]]
df3 = london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, 
                                    london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]]
df4 = london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, 
                                    london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]]
df5 = london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, 
                                    london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]]


In [61]:
# Cluster 0
df1.head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1439,2.200000e+06,Pub,Fish & Chips Shop,Sandwich Place,Performing Arts Venue,Recording Studio,Park,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market
1508,2.225000e+06,Pub,Sandwich Place,Discount Store,Hardware Store,Supermarket,Casino,Bar,Bus Station,Gym,River
2046,2.218333e+06,Grocery Store,Fried Chicken Joint,Bus Stop,Fish & Chips Shop,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm
2540,2.213750e+06,Italian Restaurant,Pub,Café,Japanese Restaurant,Asian Restaurant,English Restaurant,Juice Bar,Cocktail Bar,Grocery Store,Bakery
3230,2.240000e+06,Chinese Restaurant,Café,Hotel,Metro Station,Food Court,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant


In [62]:
# Cluster 1
df2.head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
499,2.462500e+06,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Jewelry Store,Auto Garage,Pizza Place,Pilates Studio,Pharmacy,Event Space,Exhibit
680,2.475000e+06,Pub,Construction & Landscaping,Bar,Performing Arts Venue,Park,Dance Studio,Deli / Bodega,Event Service,Event Space,Exhibit
1021,2.500000e+06,Performing Arts Venue,Restaurant,Zoo Exhibit,Fast Food Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm
2223,2.500000e+06,Italian Restaurant,Hotel,Light Rail Station,Dessert Shop,Japanese Restaurant,Café,Lebanese Restaurant,Park,Pizza Place,Deli / Bodega
2611,2.470154e+06,Pub,Lake,Fast Food Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market


In [63]:
# Cluster 2
df3.head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
989,2.344600e+06,Sandwich Place,Sports Club,Auto Garage,Locksmith,Zoo Exhibit,Farmers Market,Event Space,Exhibit,Fabric Shop,Falafel Restaurant
1326,2.376941e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2168,2.361750e+06,Pub,Café,Coffee Shop,Bus Stop,Park,Platform,Thai Restaurant,Italian Restaurant,Bar,Middle Eastern Restaurant
2395,2.323750e+06,Café,Bar,Italian Restaurant,Pub,Coffee Shop,French Restaurant,Park,Caribbean Restaurant,Pizza Place,Spa
3912,2.338185e+06,Gym / Fitness Center,Convenience Store,Sports Club,Martial Arts School,Bus Stop,Rugby Pitch,Farm,Event Service,Event Space,Exhibit


In [64]:
# Cluster 3
df4.head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
171,2.283829e+06,Restaurant,Café,Hotel,Pub,Park,Art Gallery,Plaza,Pizza Place,Coffee Shop,Breakfast Spot
186,2.292500e+06,Café,Pub,Bar,Indian Restaurant,Restaurant,Coffee Shop,Park,Seafood Restaurant,Fish & Chips Shop,New American Restaurant
258,2.283095e+06,Pub,Indian Restaurant,Playground,Lawyer,Farmers Market,English Restaurant,Event Service,Event Space,Exhibit,Fabric Shop
455,2.300000e+06,Cha Chaan Teng,Noodle House,Dessert Shop,Hotel,Sporting Goods Shop,Hong Kong Restaurant,Vegetarian / Vegan Restaurant,Cantonese Restaurant,Bakery,Toy / Game Store
2125,2.305400e+06,Pub,Ice Cream Shop,Bakery,Yoga Studio,Coffee Shop,Indian Restaurant,Hotel,Hostel,Park,Breakfast Spot


In [65]:
# Cluster 4
df5.head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
536,2425000.0,Pub,Grocery Store,Bakery,Indian Restaurant,Mediterranean Restaurant,Coffee Shop,Ice Cream Shop,Middle Eastern Restaurant,Moroccan Restaurant,Fish & Chips Shop
1179,2425000.0,Theater,Coffee Shop,Bookstore,Clothing Store,Beer Bar,Bakery,Dessert Shop,Indian Restaurant,Boutique,Sushi Restaurant
1180,2400000.0,Chinese Restaurant,Park,Grocery Store,English Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm
1220,2400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1746,2400000.0,Sandwich Place,Lounge,Health & Beauty Service,Park,Convenience Store,Food & Drink Shop,Flea Market,Empanada Restaurant,Food Court,English Restaurant


## Discussion

After studying the dataset, we may gain a better insight across the London house market based on the street average price and the amenities nearby. London Housing Market, though is said to be in a rut at the moment, is still vibrant and of potential.

We start off by analysising the result from different angles.

#### Grouping by Clusters
According to the five clusterss produced in the end, we could target housebuyers' interests when suggesting the best venues. For example, Cluster 0 may targeting customers looking for various food places and economical life expenses, for they show a most common of restaurants, pub and cafe with a relatively low average price. Instead, customers fancy high-end community may make their choices among estates on the streets in Cluster 1, with arts meseums and exhibitions surrounded.

#### Future Market Study and Prediction
In another hand, we could also make our prediction on the future market based on the level of infrastructures and amenities such as schools and hospitals. Further looking into the map, it is interesting to notice that while most venues are in West London (Chelsea, Kensinton, Marylebone), there are outstanding venues located at South-West London (Wimbledon, Baham) and North-West London (Kilburn, Hampstead), which may due to the city expansion. This may favor potential investors into the house market who want to purchase venues as an investment option. 

#### Further implementations
Further Study can also be brought out based on detailed requirements provided by the housebuyers, such as sorting the venues in each clusters by average price or picking out the estates with the most common venues specified. 

## Conclusion

Despite the fact that housing markets in London have witnessed a number of ups and downs, with volatile house prices in London tending to amplify changes in national house prices recently, there is still great potential lying in the market for investers and housebuyers. Recently as a result of the pandemic, increasing unemployment rate, wage cuts, business failures, and job uncertainty also brought huge impact to the house market in London. However, it is relatively hard to provide an insight of the market from the patterns of previous cycles for clients, so in this report we focused on the integration of data science and machine learning technics to assist clients making sensible and efficient choices. From the results provided, we could easily recommend profitable venues according to amenities and infrastructures surrounded.

We first retrieved data on London properties and the relative price from the HM Land Registry (http://landregistry.data.gov.uk/). Amenities and infrastructure dataframe is built with the Foursquare API. Merging the two datasets together, we were able to have a more detailed information about the market and start our study. K-means clustering technique is used to cluster all the data into 5 different clusters. Finally, we could use the results to better study the london house market and make our choice of data based on clients' requirment. 

This approach is an visualizable and efficient way for housebuyers to gain an insight of the house market and the methodology can be applied for other places with proper data provided. It is also worth notice that the due to the robustion of house market, making wise decision is important even with the help of algorithm technics and data analysis which is based on previous market performance but not predicting future incidences.